In [4]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

from evaluator_ANAELE import *

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, SparseCategoricalAccuracy
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import SGD

In [5]:
# FUNCTIONS

# to show performance

def evaluate(Y_pred,Y,S,will_print=1):
    '''returns model accuracy, final score, macro fscore ans TPR gap
    input : 2 np arrays of same dimension
    output : array of 4 values
    '''
    accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
    print(f"Accuracy on transformed test data: {accuracy}")
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2

    if will_print==1:
        #print results
        print('final score',final_score)
        print('macro_fscore',eval_scores['macro_fscore'])
        print('1-eval_scores[\'TPR_GAP\']',1-eval_scores['TPR_GAP'])
    
    return accuracy, final_score, eval_scores['macro_fscore'],1-eval_scores['TPR_GAP'] , eval_scores , confusion_matrices_eval

# to predict X_test and save to file

def save_X_test_true(X, model,name):
    Y_pred = model.predict(X)
    results=pd.DataFrame(y_pred, columns= ['score'])
    file_name = "Data_Challenge_MDI_341_"+str(name)+".csv"
    results.to_csv(file_name, header = None, index = None)
    
    return Y_pred

In [6]:
##############################################################
# LOAD DATA, 
#############################################################

# Load pickle file and convert to numpy array
with open('data-challenge-student.pickle', 'rb') as handle:
    # dat = pickle.load(handle)
    dat = pd.read_pickle(handle)
 
#Check keys()
print(dat.keys())
X = dat['X_train']
Y = dat['Y']
S = dat['S_train']

#create a label to distiguish 56 labels Y x 2 (man or woman)
# 0 to 27 = non sensitive group | 28 + [0 , 27] = 28 to 55 = sensitive group
Y56 = Y+28*S

X_test_true = dat['X_test']
S_test_true = dat['S_test']

# check size
print(X.shape,Y.shape,S.shape,X_test_true.shape,S_test_true.shape)


dict_keys(['X_train', 'X_test', 'Y', 'S_train', 'S_test'])
(27749, 768) (27749,) (27749,) (11893, 768) (11893,)


In [7]:
##############################################################
# train_test_split
##############################################################

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, Y56_train, Y56_test = train_test_split(X, Y56, test_size=0.2, random_state=42)
Y_train = Y56_train % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_train = Y56_train//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
Y_test = Y56_test % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_test = Y56_test//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
print('train:',X_train.shape,Y_train.shape,S_train.shape)
print('test:',X_test.shape,Y_test.shape, S_test.shape)

train: (22199, 768) (22199,) (22199,)
test: (5550, 768) (5550,) (5550,)


In [12]:
##############################################################
#  DEFINE CUSTOM LOSS FUNCTION AND EVALUATION FUNCTIONS
#   
#   soft_f1_loss
#   macro_soft_f1_loss
#   calculate_exact_macro_f1
#   calculate_class_tpr_gap
#   average_tpr_gap_per_class
#   
##############################################################


def soft_f1_loss(y_true, y_pred):
    """
    Differentiable approximation of the F1 score as a loss function.
    """
    y_pred_probs = torch.softmax(y_pred, dim=1)
    tp = torch.sum(y_true * y_pred_probs, dim=0)
    pp = torch.sum(y_pred_probs, dim=0)
    ap = torch.sum(y_true, dim=0)
    precision = tp / (pp + 1e-6)
    recall = tp / (ap + 1e-6)
    soft_f1 = 2 * (precision * recall) / (precision + recall + 1e-6)
    loss = 1 - soft_f1.mean()  # Mean to aggregate over all classes
    return loss

def soft_macro_f1_loss(y_true, y_pred):
    """
    Differentiable approximation of the macro F1 score as a loss function.
    Calculates the F1 score for each class independently and then takes the average.
    """
    y_pred_probs = torch.softmax(y_pred, dim=1)
    tp = torch.sum(y_true * y_pred_probs, dim=0)
    pp = torch.sum(y_pred_probs, dim=0)
    ap = torch.sum(y_true, dim=0)
    
    precision = tp / (pp + 1e-6)
    recall = tp / (ap + 1e-6)
    
    f1_per_class = 2 * (precision * recall) / (precision + recall + 1e-6)
    macro_f1 = torch.mean(f1_per_class)  # Average F1 score across all classes
    
    loss = 1 - macro_f1  # Minimizing loss is maximizing macro F1 score
    return loss


def macro_f1(y_true, y_pred):
    """
    Calculate the exact macro F1 score for evaluation.
    """
    y_pred_labels = torch.argmax(y_pred, dim=1)
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true
    f1 = f1_score(y_true_labels.cpu().numpy(), y_pred_labels.cpu().numpy(), average='macro')
    return f1

def tpr_gap(y_true, y_pred_probs, protected_attribute, class_idx):
    """
    Calculate the TPR gap for a specific class across protected groups.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred_probs: Tensor of predicted probabilities (after softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    - class_idx: Index of the class for which to calculate the TPR gap.
    
    Returns:
    - TPR gap for the specified class.
    """
    # Convert one-hot labels to class indices for gathering
    y_true_indices = torch.argmax(y_true, dim=1)
    
    # Calculate overall TPR for the current class
    overall_mask = y_true_indices == class_idx
    overall_tpr = torch.sum((y_pred_probs[:, class_idx] > 0.5) & overall_mask).float() / (torch.sum(overall_mask).float() + 1e-6)
    
    # Initialize list to store TPR for each protected group
    group_tprs = []
    
    # Calculate TPR for each protected group
    for group_val in protected_attribute.unique():
        group_mask = (protected_attribute == group_val) & overall_mask
        group_tpr = torch.sum((y_pred_probs[:, class_idx] > 0.5) & group_mask).float() / (torch.sum(group_mask).float() + 1e-6)
        group_tprs.append(group_tpr)
    
    # Calculate TPR gap for the current class
    tpr_gaps = torch.abs(torch.tensor(group_tprs) - overall_tpr)
    
    return torch.mean(tpr_gaps)  # Return the mean TPR gap for this class

def macro_tpr_gap(y_true, y_pred, protected_attribute):
    """
    Calculate the average TPR gap per class by calling tpr_gap for each class.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred: Tensor of predicted logits (before softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    
    Returns:
    - Average TPR gap across all classes.
    """
    # Apply softmax to get probabilities
    y_pred_probs = torch.softmax(y_pred, dim=1)
    
    # Initialize list to store TPR gaps for all classes
    class_tpr_gaps = []
    
    # Iterate over each class
    num_classes = y_true.shape[1]
    for class_idx in range(num_classes):
        class_tpr_gap = tpr_gap(y_true, y_pred_probs, protected_attribute, class_idx)
        class_tpr_gaps.append(class_tpr_gap)
    
    # Calculate the average TPR gap across all classes
    avg_tpr_gap = torch.mean(torch.stack(class_tpr_gaps))
    
    return avg_tpr_gap

def soft_final_score_loss(y_true, y_pred, protected_attribute):
    """
    Combine soft macro F1 score and TPR gap to create a final evaluation metric.
    """
    soft_macro_f1 = soft_macro_f1_loss(y_true, y_pred)  # Calculate soft macro F1 score
    macro_tpr_gap = macro_tpr_gap(y_true, y_pred, protected_attribute)  # Calculate TPR gap
    
    soft_final_score = (1 - soft_macro_f1 + (1 - tpr_gap)) / 2
    return soft_final_score

def final_score(y_true, y_pred, protected_attribute):
    """
    Combine soft macro F1 score and TPR gap to create a final evaluation metric.
    """
    macro_f1 = macro_f1(y_true, y_pred)  # Calculate macro F1 score
    macro_tpr_gap = macro_tpr_gap(y_true, y_pred, protected_attribute)  # Calculate macro TPR gap
    
    final_score = (1 - macro_f1 + (1 - macro_tpr_gap)) / 2
    return final_score


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score

# 1. Transform DataFrames into Tensors
# ------------------------------------

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.long)
S_train_tensor = torch.tensor(S_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.long)
S_test_tensor = torch.tensor(S_test.values, dtype=torch.long)

Y_train_one_hot = torch.nn.functional.one_hot(Y_train_tensor, num_classes=Y_train.nunique())

# 2. Define the model and optimizer
# ---------------------------------

model = nn.Sequential(
    nn.Linear(768, 28),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Linear(28, 28),  # Additional layer for complexity
    nn.LogSoftmax(dim=1)  # LogSoftmax for multi-class classification
)

optimizer = optim.SGD(model.parameters(), lr=0.01)

# 3. Train the model with the custom loss function final_eval
# -----------------------------------------------------------

num_epochs = 10#00  # Adjust as necessary

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs_train = model(X_train_tensor)
    loss = soft_final_score_loss(Y_train_one_hot.float(), outputs_train, S_train_tensor)
    
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        model.eval()
        with torch.no_grad():
            # Calculate metrics for training data
            outputs_train = model(X_train_tensor)
            final_score_train = final_score(Y_train_one_hot.float(), outputs_train, S_train_tensor)
            macro_f1_train = macro_f1(Y_train_one_hot.float(), outputs_train)
            inv_macro_tpr_gap_train = 1 - macro_tpr_gap(Y_train_one_hot.float(), outputs_train, S_train_tensor)
            
            # Calculate metrics for test data
            outputs_test = model(X_test_tensor)
            final_score_test = final_score(Y_test_tensor, outputs_test, S_test_tensor)
            macro_f1_test = macro_f1(Y_test_tensor, outputs_test)
            inv_macro_tpr_gap_test = 1 - macro_tpr_gap(Y_test_tensor, outputs_test, S_test_tensor)
            
            print(f'Epoch {epoch + 1}, Loss: {loss.item()}, Final Score Train: {final_score_train.item()}, Final Score Test: {final_score_test.item()}, macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}, 1-TPR Gap Train: {inv_macro_tpr_gap_train}, 1-TPR Gap Test: {inv_macro_tpr_gap_test}')

# 4. Make Predictions and Evaluate with final_score
# -------------------------------------------------
            
with torch.no_grad():
    model.eval()
    Y_pred_probs = model(X_test_tensor)
    final_score = final_score(Y_test_tensor, Y_pred_probs, S_test_tensor)
    macro_f1 = macro_f1(Y_test_tensor, outputs_test)
    inv_macro_tpr_gap = 1 - macro_tpr_gap(Y_test_tensor, outputs_test, S_test_tensor)
    print(f'Final Evaluation Score: {final_score.item()} Macro F1: {macro_f1.item()} 1-TPR_gap: { inv_macro_tpr_gap.item() }')

NameError: name 'TPR_gap' is not defined

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Convert DataFrames or arrays to PyTorch tensors if they are not already
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32) if isinstance(X_train, pd.DataFrame) else torch.from_numpy(X_train).float()
Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.int64) if isinstance(Y_train, pd.Series) else torch.from_numpy(Y_train).long()

# Convert Y_train_tensor to one-hot encoding for multi-class
Y_train_one_hot = torch.nn.functional.one_hot(Y_train_tensor, num_classes=28)


# Define the model using nn.Sequential
model = nn.Sequential(
    nn.Linear(768, 28),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Linear(28, 28),  # Additional layer for complexity
    nn.LogSoftmax(dim=1)  # LogSoftmax for multi-class classification
)

# Define an optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Training loop
num_epochs = 10000  # Example number of epochs

for epoch in range(num_epochs):
    optimizer.zero_grad()  # Zero the gradients
    
    outputs = model(X_train_tensor)  # Forward pass
    
    loss = soft_f1_loss(Y_train_one_hot.float(), outputs)  # Calculate loss using the custom F1 loss function
    
    loss.backward()  # Backward pass
    optimizer.step()  # Update weights
    
    if (epoch+1) % 1 == 0:  # Display loss every epoch
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

Epoch [1/10000], Loss: 0.9742507934570312
Epoch [2/10000], Loss: 0.9742189049720764
Epoch [3/10000], Loss: 0.9741873145103455
Epoch [4/10000], Loss: 0.9741561412811279
Epoch [5/10000], Loss: 0.9741252064704895
Epoch [6/10000], Loss: 0.9740946292877197
Epoch [7/10000], Loss: 0.9740644097328186
Epoch [8/10000], Loss: 0.9740345478057861
Epoch [9/10000], Loss: 0.9740051031112671
Epoch [10/10000], Loss: 0.9739761352539062
Epoch [11/10000], Loss: 0.9739474058151245
Epoch [12/10000], Loss: 0.9739190936088562
Epoch [13/10000], Loss: 0.9738910794258118
Epoch [14/10000], Loss: 0.9738634824752808
Epoch [15/10000], Loss: 0.9738361835479736
Epoch [16/10000], Loss: 0.9738092422485352
Epoch [17/10000], Loss: 0.9737827181816101
Epoch [18/10000], Loss: 0.9737566113471985
Epoch [19/10000], Loss: 0.9737308025360107
Epoch [20/10000], Loss: 0.9737053513526917
Epoch [21/10000], Loss: 0.9736801385879517
Epoch [22/10000], Loss: 0.9736553430557251
Epoch [23/10000], Loss: 0.973630964756012
Epoch [24/10000], Los

In [36]:
import pandas as pd
from sklearn.metrics import classification_report
import torch

# Assuming model is already trained and X_test is a DataFrame

# Convert X_test to a PyTorch tensor
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

# Make predictions
with torch.no_grad():  # We do not need gradient computation for prediction
    model.eval()  # Set the model to evaluation mode
    Y_pred_probs = model(X_test_tensor)
    Y_pred = torch.argmax(Y_pred_probs, dim=1)  # Get the class with the highest probability

# Convert Y_pred to a DataFrame
Y_pred_df = pd.DataFrame(Y_pred.numpy(), columns=['Predicted'])

# Evaluate Y_pred compared to Y_test (assuming Y_test is a numpy array or a pandas Series)
print(classification_report(Y_test, Y_pred_df['Predicted']))

# If you want to use the exact F1 score for evaluation, you can directly use it from sklearn.metrics
from sklearn.metrics import f1_score
print("Exact F1 Score (micro):", f1_score(Y_test, Y_pred_df['Predicted'],average = 'micro'))  # 'weighted' for multi-class
print("Exact F1 Score (macro):", f1_score(Y_test, Y_pred_df['Predicted'], average='macro'))  # 'weighted' for multi-class

# Returning Y_pred as a DataFrame makes sense for further analysis or submission
#return Y_pred_df

              precision    recall  f1-score   support

           0       0.71      0.56      0.62        81
           1       0.61      0.55      0.58       127
           2       0.82      0.87      0.84       458
           3       0.00      0.00      0.00        36
           4       0.84      0.67      0.74        48
           5       0.79      0.78      0.78        72
           6       0.90      0.73      0.81       178
           7       0.80      0.69      0.74        54
           8       0.92      0.67      0.77        18
           9       0.78      0.79      0.79        91
          10       0.00      0.00      0.00        22
          11       0.63      0.77      0.69       286
          12       0.88      0.69      0.78       110
          13       0.76      0.73      0.75       258
          14       0.78      0.78      0.78       112
          15       0.00      0.00      0.00        19
          16       0.57      0.52      0.54        33
          17       0.77    

/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**2. REGRESSION WITH CUSTOM LOSS macro F1**
---

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
import numpy as np

# Assuming model, optimizer, X_train_tensor, Y_train_one_hot, X_test_tensor, Y_test are already defined

# Convert Y_test to one-hot encoding if it's not already one-hot encoded
# This is necessary for consistency in our loss function calculations
Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.int64) if isinstance(Y_test, pd.Series) else torch.from_numpy(Y_test).long()
Y_test_one_hot = torch.nn.functional.one_hot(Y_test_tensor, num_classes=28)


# Define the model using nn.Sequential
model = nn.Sequential(
    nn.Linear(768, 28),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Linear(28, 28),  # Additional layer for complexity
    nn.LogSoftmax(dim=1)  # LogSoftmax for multi-class classification
)

# Define an optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)

num_epochs = 10000  # Example number of epochs

for epoch in range(num_epochs):
    optimizer.zero_grad()  # Zero the gradients
    
    # Forward pass on the training data
    outputs_train = model(X_train_tensor)
    loss_train = macro_soft_f1_loss(Y_train_one_hot.float(), outputs_train)
    
    # Backward pass and optimize
    loss_train.backward()
    optimizer.step()
    
    # No gradient computation needed for evaluation
    with torch.no_grad():
        model.eval()  # Set the model to evaluation mode
        
        # Forward pass on the validation data
        outputs_test = model(X_test_tensor)
        
        # Calculate the exact macro F1 score for both training and validation data
        f1_train = calculate_exact_macro_f1(Y_train_one_hot.float(), outputs_train)
        f1_test = calculate_exact_macro_f1(Y_test_one_hot.float(), outputs_test)
        
        model.train()  # Set the model back to training mode
    
    # Print loss and F1 score
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss_train.item():.4f}, macro F1 Train: {f1_train:.4f}, macro F1 Test: {f1_test:.4f}')

Epoch [1/10000], Loss: 0.3177, macro F1 Train: 0.7090, macro F1 Test: 0.6349
Epoch [2/10000], Loss: 0.3177, macro F1 Train: 0.7090, macro F1 Test: 0.6349
Epoch [3/10000], Loss: 0.3177, macro F1 Train: 0.7090, macro F1 Test: 0.6349
Epoch [4/10000], Loss: 0.3177, macro F1 Train: 0.7090, macro F1 Test: 0.6349
Epoch [5/10000], Loss: 0.3177, macro F1 Train: 0.7090, macro F1 Test: 0.6349
Epoch [6/10000], Loss: 0.3177, macro F1 Train: 0.7090, macro F1 Test: 0.6349
Epoch [7/10000], Loss: 0.3177, macro F1 Train: 0.7090, macro F1 Test: 0.6349
Epoch [8/10000], Loss: 0.3176, macro F1 Train: 0.7091, macro F1 Test: 0.6351
Epoch [9/10000], Loss: 0.3176, macro F1 Train: 0.7091, macro F1 Test: 0.6351
Epoch [10/10000], Loss: 0.3176, macro F1 Train: 0.7091, macro F1 Test: 0.6351
Epoch [11/10000], Loss: 0.3176, macro F1 Train: 0.7095, macro F1 Test: 0.6351
Epoch [12/10000], Loss: 0.3176, macro F1 Train: 0.7095, macro F1 Test: 0.6351
Epoch [13/10000], Loss: 0.3176, macro F1 Train: 0.7095, macro F1 Test: 0.

In [41]:
import pandas as pd
from sklearn.metrics import classification_report
import torch

# Assuming model is already trained and X_test is a DataFrame

# Convert X_test to a PyTorch tensor
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

# Make predictions
with torch.no_grad():  # We do not need gradient computation for prediction
    model.eval()  # Set the model to evaluation mode
    Y_pred_probs = model(X_test_tensor)
    Y_pred = torch.argmax(Y_pred_probs, dim=1)  # Get the class with the highest probability

# Convert Y_pred to a DataFrame
Y_pred_df = pd.DataFrame(Y_pred.numpy(), columns=['Predicted'])

# Evaluate Y_pred compared to Y_test (assuming Y_test is a numpy array or a pandas Series)
print(classification_report(Y_test, Y_pred_df['Predicted']))

# If you want to use the exact F1 score for evaluation, you can directly use it from sklearn.metrics
from sklearn.metrics import f1_score
print("Exact F1 Score (micro):", f1_score(Y_test, Y_pred_df['Predicted'],average = 'micro'))  # 'weighted' for multi-class
print("Exact F1 Score (macro):", f1_score(Y_test, Y_pred_df['Predicted'], average='macro'))  # 'weighted' for multi-class

# Returning Y_pred as a DataFrame makes sense for further analysis or submission
#return Y_pred_df

              precision    recall  f1-score   support

           0       0.72      0.58      0.64        81
           1       0.63      0.57      0.60       127
           2       0.81      0.86      0.84       458
           3       0.00      0.00      0.00        36
           4       0.82      0.69      0.75        48
           5       0.77      0.78      0.77        72
           6       0.89      0.75      0.81       178
           7       0.83      0.70      0.76        54
           8       1.00      0.56      0.71        18
           9       0.80      0.78      0.79        91
          10       0.00      0.00      0.00        22
          11       0.64      0.76      0.70       286
          12       0.86      0.67      0.76       110
          13       0.76      0.76      0.76       258
          14       0.82      0.76      0.79       112
          15       0.00      0.00      0.00        19
          16       0.48      0.48      0.48        33
          17       0.73    

/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/anaele/myenv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**CUSTON LOSS FUNCTION TRP GAP**
---

In [ ]:
import torch

def gap_TPR(y_true, y_pred, protected_attribute):
    """
    Calculate the average TPR gap for each class across protected groups.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred: Tensor of predicted logits (before softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    
    Returns:
    - Average TPR gap across all classes.
    """
    # Apply softmax to get probabilities
    y_pred_probs = torch.softmax(y_pred, dim=1)
    
    # Convert one-hot labels to class indices for gathering
    y_true_indices = torch.argmax(y_true, dim=1)
    
    # Initialize TPR storage
    tpr_gaps = []
    
    # Iterate over each class
    num_classes = y_true.shape[1]
    for class_idx in range(num_classes):
        # Calculate TPR for the current class across all groups
        tpr_list = []
        
        # Calculate overall TPR for the current class
        overall_mask = y_true_indices == class_idx
        overall_tpr = torch.sum((y_pred_probs[:, class_idx] > 0.5) & overall_mask).float() / torch.sum(overall_mask).float()
        
        # Calculate TPR for each protected group
        for group_val in protected_attribute.unique():
            group_mask = (protected_attribute == group_val) & overall_mask
            group_tpr = torch.sum((y_pred_probs[:, class_idx] > 0.5) & group_mask).float() / torch.sum(group_mask).float()
            tpr_list.append(group_tpr)
        
        # Calculate TPR gap for the current class and store it
        tpr_gaps.append(torch.abs(torch.tensor(tpr_list) - overall_tpr))
    
    # Calculate the average TPR gap across all classes
    avg_tpr_gap = torch.mean(torch.stack(tpr_gaps))
    
    return avg_tpr_gap

In [34]:
# définir une fonction de loss personalisée sur F1 score

def custom_loss(y_true, y_pred):
    # Seuils pour décider des classifications
    y_pred = K.round(y_pred)
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    
    f1 = 2* (precision*recall) / (precision + recall + K.epsilon())
    return 1 - K.mean(f1)  # Moyenne du F1 sur le batch

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim

# Définir le modèle en utilisant nn.Sequential
model = nn.Sequential(
    nn.Linear(768, 28),  # Couche linéaire avec 768 entrées et 28 sorties (classes)
    nn.LogSoftmax(dim=1)  # LogSoftmax pour obtenir des probabilités log sur la dimension des classes
)
print(model)

# Définir un optimiseur
optimizer = optim.SGD(model.parameters(), lr=0.01)
num_epochs =1

# Boucle d'entraînement
for epoch in range(num_epochs):  
    
    optimizer.zero_grad()  # Zéro aux gradients
    
    # forward pass
    outputs = model(X_train)  # X_train doit être votre tensor de données d'entrée
    
    # calcul de la perte
    loss = custom_loss(outputs, Y_train)  # Y_train doit être votre tensor de cibles
    
    # backward pass
    loss.backward()  # calcul des gradients
    
    # mise à jour des poids
    optimizer.step()
    
    if (epoch+1) % 100 == 0:  # Affichage toutes les 100 époques
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

loss

Sequential(
  (0): Linear(in_features=768, out_features=28, bias=True)
  (1): LogSoftmax(dim=1)
)


2024-03-10 22:49:54.308305: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 22:49:54.309295: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.